<a href="https://colab.research.google.com/github/plushvoxel/Project-Lernende-Agenten-colab/blob/master/CNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np
import tensorflow as tf
from tarfile import open as taropen
from urllib import request
from struct import unpack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [0]:
def create_data(trainP = 60, valP= 20, testP = 20):
    pre: trainP + valP + testP <= 100
    DataBaseURL= "https://marble.adhara.uberspace.de/"
    Classes = ["fm", "pager", "smartwares"]
    data = []
    normalisationFactor = 100
    test = 0

    for modulation in Classes:
        filename = "{}.tar".format(modulation)
        request.urlretrieve("https://marble.adhara.uberspace.de/{}_512.tar".format(modulation), filename)
        tar = taropen(filename)
        class_label = Classes.index(modulation)
        for member in tar.getmembers():
            test = test + 1
            with tar.extractfile(member) as f:
                sample = []
                buffer = f.read()
                num_floats = len(buffer)//4
                floats = unpack("f"*num_floats, buffer)
                i = floats[0::2][::4]
                q = floats[1::2][::4]
                for j in range(min(len(i), len(q))):
                    #here happens the scaling or whatever you want
                    sample.append([i[j]/normalisationFactor, q[j]/normalisationFactor])
                #here happens some cross_feature if you want. just append it to the sample
                
                data.append([np.array(sample), class_label])

    random.shuffle(data)
    #split all data into train, val and test
    size =len(data)
    trainSize = (size * trainP)//100
    valSize = (size * valP)//100
    testSize = (size * testP)//100
    train = data[:trainSize]
    val = data[trainSize: trainSize + valSize]
    test = data[trainSize + valSize: trainSize + valSize + testSize]
    x_train = []
    y_train = []
    x_val = []
    y_val = []
    x_test = []
    y_test = []
    for sample, target in train:
      x_train.append(sample)
      y_train.append(target)
    for sample, target in val:
      x_val.append(sample)
      y_val.append(target)
    for sample, target in test:
      x_test.append(sample)
      y_test.append(target)
    
    
    return np.array(x_train), np.array(x_val), np.array(x_test), y_train, y_val, y_test 

In [0]:
x_train, x_val, x_test, ytrain, y_val, y_test = create_data()


In [32]:
print(np.max(x_train[0]))

0.3398982238769531


In [33]:
print(x_train.shape)

(19704, 128, 2)


In [0]:
training_iters = 200 
learning_rate = 0.001 
batch_size = 128

#data input (img shape: 128*2)
n_input_1 = 128
n_input_2 = 2

# data total classes (0-9 digits)
n_classes = 3

#both placeholders are of type float
x = tf.placeholder("float", [None, n_input_1,n_input_2,1])
y = tf.placeholder("float", [None, n_classes])

In [0]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')